# ResNet50 implementation using CIFAR10 dataset

In [28]:
import numpy as np
import keras
from keras import layers
import keras.backend as K
from keras.datasets import cifar10
from keras.utils import np_utils
from keras.callbacks import CSVLogger, EarlyStopping
from keras.initializers import glorot_uniform
import cv2

K.set_image_data_format('channels_last')
K.set_learning_phase(1)

In [49]:
(X_train, Y_train), (X_test, Y_test) = cifar10.load_data()

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

X_train = np.array([cv2.resize(img, (64,64)) for img in X_train[:10000,:,:,:]])
X_test = np.array([cv2.resize(img, (64,64)) for img in X_test[:1000,:,:,:]])

n_classes = 10

Y_train = np_utils.to_categorical(Y_train[:10000,:], n_classes)
Y_test = np_utils.to_categorical(Y_test[:1000,:], n_classes)

mean_image = np.mean(X_train, axis=0)
X_train -= mean_image
X_test -= mean_image
X_train /= 128.
X_test /= 128.

In [50]:
print X_train.shape
print Y_train.shape
print X_test.shape
print Y_test.shape

(10000, 64, 64, 3)
(10000, 10)
(1000, 64, 64, 3)
(1000, 10)


In [10]:
def conv_block(input_x, kernels, filter_sizes, strides):
    input_shortcut = input_x
    
    k1, k2, k3 = kernels
    f1, f2, f3 = filter_sizes
    s1, s2, s3 = strides
    
    X = layers.Conv2D(k1, kernel_size=(f1,f1), strides=(s1,s1), padding="valid", kernel_initializer = glorot_uniform(seed=0))(input_x)
    X = layers.BatchNormalization(axis=3)(X)
    X = layers.Activation('relu')(X)
    
    X = layers.Conv2D(k2, kernel_size=(f2,f2), strides=(s2,s2), padding="same", kernel_initializer = glorot_uniform(seed=0))(X)
    X = layers.BatchNormalization(axis=3)(X)
    X = layers.Activation('relu')(X)

    X = layers.Conv2D(k3, kernel_size=(f3,f3), strides=(s3,s3), padding="valid", kernel_initializer = glorot_uniform(seed=0))(X)
    X = layers.BatchNormalization(axis=3)(X)
    
    input_shortcut = layers.Conv2D(k3, kernel_size=(f3, f3), strides=(s1,s1), padding="valid", kernel_initializer=glorot_uniform(seed=0))(input_shortcut)
    input_shortcut = layers.BatchNormalization()(input_shortcut)
    
    X = layers.Add()([input_shortcut,X])
    X = layers.Activation('relu')(X)
    
    return X

In [11]:
def identity_block(input_x, kernels, filter_sizes, strides):
    input_shortcut = input_x
    
    k1, k2, k3 = kernels
    f1, f2, f3 = filter_sizes
    s1, s2, s3 = strides
    
    X = layers.Conv2D(k1, kernel_size=(f1,f1), strides=(s1,s1), padding="valid", kernel_initializer = glorot_uniform(seed=0))(input_x)
    X = layers.BatchNormalization(axis=3)(X)
    X = layers.Activation('relu')(X)
    
    X = layers.Conv2D(k2, kernel_size=(f2,f2), strides=(s2,s2), padding="same", kernel_initializer = glorot_uniform(seed=0))(X)
    X = layers.BatchNormalization(axis=3)(X)
    X = layers.Activation('relu')(X)

    X = layers.Conv2D(k3, kernel_size=(f3,f3), strides=(s3,s3), padding="valid", kernel_initializer = glorot_uniform(seed=0))(X)
    X = layers.BatchNormalization(axis=3)(X)
    
    X = layers.Add()([input_shortcut,X])
    X = layers.Activation('relu')(X)
    
    return X
    

In [43]:
def ResNet50(input_shape, n_classes):
    
    input_x = layers.Input(input_shape)
    
    X = layers.ZeroPadding2D((3,3))(input_x)
    
    X = layers.Conv2D(64, kernel_size=(7,7), strides=(2,2), padding="same", kernel_initializer = glorot_uniform(seed=0))(input_x)
    X = layers.BatchNormalization(axis=3)(X)
    X = layers.Activation('relu')(X)
    X = layers.MaxPooling2D(pool_size=(3,3), strides=(2,2))(X)
    
    X = conv_block(X, [64, 64, 256], [1, 3, 1], [1, 1, 1])
    X = identity_block(X, [64, 64, 256], [1, 3, 1], [1, 1, 1])
    X = identity_block(X, [64, 64, 256], [1, 3, 1], [1, 1, 1])
    
    X = conv_block(X, [128, 128, 512], [1, 3, 1], [2, 1, 1])
    X = identity_block(X, [128, 128, 512], [1, 3, 1], [1, 1, 1])
    X = identity_block(X, [128, 128, 512], [1, 3, 1], [1, 1, 1])
    X = identity_block(X, [128, 128, 512], [1, 3, 1], [1, 1, 1])
    
    X = conv_block(X, [256, 256, 1024], [1, 3, 1], [2, 1, 1])
    X = identity_block(X, [256, 256, 1024], [1, 3, 1], [1, 1, 1])
    X = identity_block(X, [256, 256, 1024], [1, 3, 1], [1, 1, 1])
    X = identity_block(X, [256, 256, 1024], [1, 3, 1], [1, 1, 1])
    X = identity_block(X, [256, 256, 1024], [1, 3, 1], [1, 1, 1])
    X = identity_block(X, [256, 256, 1024], [1, 3, 1], [1, 1, 1])
    
    X = conv_block(X, [512, 52, 2048], [1, 3, 1], [2, 1, 1])
    X = identity_block(X, [512, 52, 2048], [1, 3, 1], [1, 1, 1])
    X = identity_block(X, [512, 52, 2048], [1, 3, 1], [1, 1, 1])
    
    #pool_size = (7,7) in orignal implementation
    X = layers.AveragePooling2D(pool_size=(2,2), strides=(1,1))(X)
    
    X = layers.Flatten()(X)
    X = layers.Dense(n_classes, activation='softmax', kernel_initializer = glorot_uniform(seed=0))(X)
    
    resnet50_model = keras.models.Model(inputs=input_x, outputs=X, name='resnet50')
    
    return resnet50_model
    

In [51]:
#input_shape of (224,224,3) in orignal implementation
resnet50_model = ResNet50(input_shape = (64, 64, 3), n_classes=10)

In [52]:
resnet50_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [53]:
resnet50_model.fit(X_train, Y_train, epochs = 3, batch_size = 32)

Epoch 1/3
10000/10000 [==============================] - 500s 50ms/step - loss: 2.7489 - acc: 0.1876
Epoch 2/3
10000/10000 [==============================] - 493s 49ms/step - loss: 2.4075 - acc: 0.2772
Epoch 3/3
10000/10000 [==============================] - 493s 49ms/step - loss: 2.4595 - acc: 0.2809
